# Exercise 5. Serial hybrid model for CSTR

Let' say we have the following reactor model:

<img src="CSTR.png" alt="CSTR" style="width: 200px;"/>

\begin{align}
\frac{\mathrm{d}c_A}{\mathrm{d}t} &= \frac{c_{A,in} - c_A}{\tau} - r\\
\frac{\mathrm{d}c_B}{\mathrm{d}t} &= \frac{c_{B,in} - c_B}{\tau} - r\\
\frac{\mathrm{d}c_X}{\mathrm{d}t} &= \frac{c_{X,in} - c_X}{\tau} + r,
\end{align}

where $c_i$ denotes the molar concentration of substance $i$ and $r$ is the reaction rate. We assume that the reactor is ideally mixed, that it has a constant volume and that the average residence time is $\tau=100\,\mathrm{1/s}$. The inlet concentrations of substance $i$ into the reactor are given by

\begin{align}
c_{A,in} &= 0.7\,\mathrm{kmol/m^3}\\
c_{B,in} &= 0.3\,\mathrm{kmol/m^3}\\
c_{X,in} &= 0\,\mathrm{kmol/m^3}.
\end{align}

For solving the differential equation system, we also need to know the initial concentration of each substance inside the reactor, denoted as $c_{i,0}$. From the data it is known that the experiment was started with the following initial concentrations:

\begin{align}
c_{A,0} &= 0.5\,\mathrm{kmol/m^3}\\
c_{B,0} &= 0.5\,\mathrm{kmol/m^3}\\
c_{X,0} &= 0\,\mathrm{kmol/m^3}.
\end{align}

Time-dependent measurements for all concentrations are available.

The task is to construct a serial hybrid model, in which the reaction rate $r$ is described by a data-driven model. In this particular example, a neural network will be used.

# Problem description

The problem with constructing a serial hybrid model, as exemplified above, is that the data-driven model cannot be trained independently of the differential equation system that describes the system (mechanistic part of the hybrid model). Actually, the neural network should learn the function $r=f(c)$ but, of course, we do not have values for $r$ which we could use to train the model. One way to solve this problem is by using sensitivity equations. This approach will be discussed in the following.

# General solution procedure

We will first generalize the differential equation system as follows:

\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= f(y, u,\phi(y,w))
\end{align}

where $y$ are states of the system, $u$ are system inputs and $\phi(y,w)$ denotes a data-driven model that describes a part of the mechanistic model with the use of the states $y$ and a set of parameter values $w$. In the case of a neural network, the parameter vector $w$ includes all the weights and biases.

In order to train any kind of model, we need a loss or cost function (usually denoted by $J$) to estimate the quality of our model predictions. Often times, the sum of squares of the deviations between the model predictions and the data is used as an objective for parameter estimation:

\begin{align}
J = 0.5 \sum_{i=1}^N\left(y_i - y_{i, \text{exp}}\right)^2,
\end{align}

where $y$ are the set of model predictions corresponding to the $N$ measurement points $y_\text{exp}$. The model predictions are depend on $u$ and $w$, i.e. $y=f(u,w)$.

Normally, when training a neural network, the gradient of the loss function with respect to the network parameters is used for optimization. The gradient of the loss function with respect to a single parameter $w_j$ is given by

\begin{align}
\frac{\partial J}{\partial w_j} = \sum_{i=1}^N\left(y_i - y_{i, \text{exp}}\right)\frac{\partial y}{\partial w_j}.
\end{align}

As can be seen, the gradient depends on $\frac{\partial y}{\partial w_j}$, i.e. the sensitivities of the system states with respect to the parameters (neural network weights and biases). Training of the neural network is then achieved by iteratively updating the parameters according to

\begin{align}
w_j^{n+1} = w_j^{n} - g \frac{\partial J}{\partial w_j^{n}},
\end{align}

where $w_j^{n+1}$ is the updated parameter that is calculated from the current parameter $w_j^{n}$ using the gradient $\frac{\partial J}{\partial w_j^{n}}$ and a learning rate $g$.

As can be seen, the problem could be solved easily, if we would have the sensitivities $\frac{\partial y}{\partial w_j}$. Since the mechanistic part of the model is given by a differential equation system, calculating these sensitivities is a bit more complicated than for an algebraic model, but luckily they are available. Using local sensitivity analysis of the ODE system and denoting the sensitivities $\frac{\partial y}{\partial w_j}$ as $s_j$, the sensitivities can be described by

\begin{align}
\frac{\mathrm{d}s_j}{\mathrm{d}t} = \frac{\partial f}{\partial y}s_j + \frac{\partial f}{\partial w_j}.
\end{align}

As can be seen, the equation above is a differential equation for the sensitivities. Since $f$ depends on the system states, the differential equations for the sensitivities need to be integrated simultaneously with the ODE system of the mechanistic part.

# Solution to the example

For the reactor example above, the loss function is defined as

\begin{align}
J = 0.5\left[ \sum_{i=1}^N\left(c_{A,i} - c_{A,i,\text{exp}}\right)^2 + \sum_{i=1}^N\left(c_{B,i} - c_{B,i,\text{exp}}\right)^2 + \sum_{i=1}^N\left(c_{X,i} - c_{X,i,\text{exp}}\right)^2\right],
\end{align}

if we assume that we have measurements for all components at each point $i$. Then, the gradient of the loss function with respect to the parameters $w$ is given as 

\begin{align}
\frac{\partial J}{\partial w_j} = \sum_{i=1}^N\left(c_{A,i} - c_{A,i,\text{exp}}\right)\frac{\partial c_{A,i}}{\partial w_j} + \sum_{i=1}^N\left(c_{B,i} - c_{B,i,\text{exp}}\right)\frac{\partial c_{B,i}}{\partial w_j} + \sum_{i=1}^N\left(c_{X,i} - c_{X,i,\text{exp}}\right)\frac{\partial c_{X,i}}{\partial w_j},
\end{align}

since the concentrations $c_A$, $c_B$ and $c_X$ are all functions of the parameters of the neural network (weights and biases). The sensitivities are calculated according to the following ODE system for each parameter $w_j$:

\begin{align}
\frac{\mathrm{d}}{\mathrm{d}t}\begin{bmatrix}
  \frac{\partial c_{A}}{\partial w_j}\\
  \frac{\partial c_{B}}{\partial w_j}\\
  \frac{\partial c_{X}}{\partial w_j}\\
\end{bmatrix}
=
\begin{bmatrix}
  -\frac{1}{\tau}-\frac{\partial r}{\partial c_A} & -\frac{\partial r}{\partial c_B} & -\frac{\partial r}{\partial c_X}\\
  -\frac{\partial r}{\partial c_A} & -\frac{1}{\tau}-\frac{\partial r}{\partial c_B} & -\frac{\partial r}{\partial c_X} \\
  +\frac{\partial r}{\partial c_A} & +\frac{\partial r}{\partial c_B} & -\frac{1}{\tau}+\frac{\partial r}{\partial c_X} \\
\end{bmatrix}
\begin{bmatrix}
  \frac{\partial c_{A}}{\partial w_j}\\
  \frac{\partial c_{B}}{\partial w_j}\\
  \frac{\partial c_{X}}{\partial w_j}\\
\end{bmatrix}
+
\begin{bmatrix}
  -\frac{\partial r}{\partial w_j}\\
  -\frac{\partial r}{\partial w_j}\\
  +\frac{\partial r}{\partial w_j}\\
\end{bmatrix}
\end{align}

Hence, for $p$ parameters and $l$ system states, there are $p*l$ sensitivity equations to be integrated additionally to the $l$ system equations. If a neural network with 3 input nodes, 10 hidden nodes and 1 output node is used, the total number of parameters is $3*10+10+10*1+1=51$ parameters for three states ($c_A$, $c_B$ and $c_X$). Thus, there would be $3*51=153$ sensitivity equations to be integrated, which quickly becomes computationally intensive.

In [1]:
# autograd is used for exact, automatic differentiation
!pip install autograd
import autograd.numpy as np
from autograd.scipy.integrate import odeint
import autograd.numpy.random as npr
from autograd import jacobian
from autograd.misc.optimizers import adam
import matplotlib.pyplot as plt
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
################################################################################
###           Create neural network for reaction rate prediction             ###
################################################################################

# initialize random number generator for repeatable results
rs = npr.RandomState(0)

def init_random_params(layer_sizes, scale):
    """Build a list of weights and biases, one for each layer in the net.
    layers is a list with the number of nodes in each layer. Minimum number of layers is three (input, hidden 
    and output) scale is a constant factor to scale the random values (down or up) if necessary"""
    params = []
    for idx in range(len(layer_sizes)-1):
        weight_mat_elem = layer_sizes[idx]*layer_sizes[idx+1]
        bias_vec_elem = layer_sizes[idx+1]
        params = np.append(params, rs.randn(weight_mat_elem+bias_vec_elem))
    return params*scale


In [3]:
#lets take some random values of some variables to understand the forward pass of neural network
layer_sizes =[3,3,1]
scale = 0.0005
init_params = init_random_params(layer_sizes, scale)

In [4]:
def neural_net_predict(params, inputs):
    """Implements a (deep) neural network for regression.
       params is a list of weights and biases.
       inputs is a matrix of input values.
       returns network prediction."""
    # Make sure that params is a vector
    params = params.flatten()
    # set separator value for easier indexing of the parameters and assigning them to weights and biases 
    # for each layer
    sep = 0
    # loop over all layers
    for idx in range(len(layer_sizes)-1):
        # calculate weight matrix
        W = params[sep:sep+layer_sizes[idx]*layer_sizes[idx+1]].reshape(layer_sizes[idx],layer_sizes[idx+1])
        # calculate bias vector
        b = params[sep+layer_sizes[idx]*layer_sizes[idx+1]:sep+layer_sizes[idx]*layer_sizes[idx+1]
                   +layer_sizes[idx+1]]
        # set new separator value
        sep = layer_sizes[idx]*layer_sizes[idx+1]+layer_sizes[idx+1]
        # calculate output as weighted sum of inputs plus bias
        outputs = np.dot(inputs, W) + b
        # apply activation function and assign the result as the input to the next layer 
        # (note that this has no effect on the output layer)
        inputs = 1/(1 + np.exp(-outputs))
    return outputs

In [5]:
# initial conditions for the concentrations in the reactor
c_A0 = 0.5
c_B0 = 0.5
c_X0 = 0
c0 = [c_A0, c_B0, c_X0]
outputs = neural_net_predict(init_params, c0)
outputs

array([0.00049865])

In [6]:
################################################################################
###                 Generate complete serial hybrid model                    ###
################################################################################

# system parameters
tau = 100

# inlet concentrations
c_Ain = 0.7
c_Bin = 0.3
c_Xin = 0

# initial conditions for the concentrations in the reactor
c_A0 = 0.5
c_B0 = 0.5
c_X0 = 0

# end time for integration
t_end = 100
n_samples = 30
t_span = np.linspace(0, t_end, n_samples)
t_span

array([  0.        ,   3.44827586,   6.89655172,  10.34482759,
        13.79310345,  17.24137931,  20.68965517,  24.13793103,
        27.5862069 ,  31.03448276,  34.48275862,  37.93103448,
        41.37931034,  44.82758621,  48.27586207,  51.72413793,
        55.17241379,  58.62068966,  62.06896552,  65.51724138,
        68.96551724,  72.4137931 ,  75.86206897,  79.31034483,
        82.75862069,  86.20689655,  89.65517241,  93.10344828,
        96.55172414, 100.        ])

In [7]:
# define system equations
def dcdt(c, t, params):
    """Mechanistic part of the hybrid model (ODE system describing the time-dependent 
    concentrations in the reactor)"""
    # disassemble input vector
    c_A, c_B, c_X = c
    # calculate reaction rates by neural network prediction
    r = neural_net_predict(params, c)
    #r = 0.08*c_A**0.7*c_B**1.3 # true underlying reaction rate
    # system equations
    dcdt = [(c_Ain-c_A)/tau - r,
            (c_Bin-c_B)/tau - r,
            (c_Xin-c_X)/tau + r]
    return np.array(dcdt)

In [8]:
# calculate system jacobian and parameter derivatives by automatic differentiation with autograd
dfdc = jacobian(dcdt, 0)    # system jacobian
dfdp = jacobian(dcdt, 2)    # parameter derivatives

In [9]:
# differential equation system
def DiffEqs(y, t, params):
    """Hybrid model including the ODE system for the concentrations as well as the sensitivities 
    that are used for training the neural network part of the model"""
    # disassemble input vector
    c = y[:3]
    s = y[3:]
    # evaluate system jacobian at current point
    dfdc_eval = dfdc(c, t, params)
    # evaluate parameter derivatives at current point
    dfdp_eval = dfdp(c, t, params) # Shape: (3, 1, 16)
    #print(dfdp_eval.shape)
    # define sensitivities for all parameters
    dsdt = np.zeros(len(s)) # preallocate memory for sensitivities
    for i in range(len_p):  # loop over all parameters to construct the corresponding sensitivity equations
        dsdt[i*len_c:(i+1)*len_c] = (dfdc_eval @ s[i*len_c:(i+1)*len_c]).flatten() + dfdp_eval[:,0,i] 
        # construct sensitivities (see https://docs.sciml.ai/v4.0/analysis/sensitivity.html#Example-solving-an-
        # ODELocalSensitivityProblem-1)
        # [c1/w1, c2/w1, c3/w1, c1/w2, ...]
    return np.concatenate((dcdt(c, t, params).flatten(), dsdt))

In [10]:
# Training parameters
scale = 0.0005
num_epochs = 1000
step_size = 0.001

# set neural network size
layer_sizes = [3, 3, 1] # no. of nodes in input layer, hidden layer(s) and output layer

# initialize parameter vector for neural network or load saved parameters
init_params = init_random_params(layer_sizes, scale)

In [11]:
# assemble initial value vector
c0 = [c_A0, c_B0, c_X0]
len_c = len(c0)
len_p = len(init_params)
s0 = np.zeros((len_p*len_c))
y0 = np.concatenate((c0,s0))

# load some simulated(fake) experimental data (see above)
c_exp = np.array(pd.read_csv('/content/ODE-data.txt', sep=';'))

In [12]:
def objective(params, iter):
    """Objective function (sum of squared errors between measurements and model predictions)"""
    # calculate hybrid model in forward direction with odeint
    sol = odeint(DiffEqs, y0, t_span, args=(params,))
    # disassemble results
    c_pred = sol[:,:3] # predicted concentrations
    return np.trace((c_pred - c_exp).T @ (c_pred - c_exp))

In [13]:
def objective_grad(params, iter):
    """Function calculates the gradient of the objective function with respect to the network parameters"""
    sol = odeint(DiffEqs, y0, t_span, args=(params,))
    # disassemble results
    c_pred = sol[:,:3] # predicted concentrations
    sens = sol[:,3:]   # sensititvities 16*3=48 -> c1/w1, c2/w1, c3/w1, c1/w2.....
    # calculate gradients of the loss function
    loss_grad = np.zeros(len_p) # set vector size
    for comp_idx in range(len_c):
        # For loop is running for each concentration and all parameters
        loss_grad += sens[:,comp_idx::3].T @ (c_pred[:,comp_idx] - c_exp[:,comp_idx])      # : is for all 30 time steps
    return loss_grad

In [14]:
def summary(params, iter, gradient):
    """Callback function gives informative output during optimization"""
    if iter % 10 == 0:
        print('step {0:5d}: {1:1.3e}'.format(iter, objective(params, iter)))
        np.save('Params', params)

In [15]:
# Optimize the network parameters
optimized_params = adam(objective_grad, init_params, step_size=step_size, num_iters=num_epochs, callback=summary)

step     0: 3.486e+00
step    10: 6.437e-01
step    20: 5.709e-01
step    30: 3.809e-01
step    40: 3.878e-01
step    50: 3.750e-01
step    60: 3.728e-01
step    70: 3.698e-01
step    80: 3.698e-01
step    90: 3.692e-01
step   100: 3.686e-01
step   110: 3.682e-01
step   120: 3.677e-01
step   130: 3.672e-01
step   140: 3.667e-01
step   150: 3.662e-01
step   160: 3.656e-01
step   170: 3.650e-01
step   180: 3.643e-01
step   190: 3.636e-01
step   200: 3.628e-01
step   210: 3.620e-01
step   220: 3.611e-01
step   230: 3.602e-01
step   240: 3.591e-01
step   250: 3.580e-01
step   260: 3.568e-01
step   270: 3.555e-01
step   280: 3.541e-01
step   290: 3.526e-01
step   300: 3.509e-01
step   310: 3.491e-01
step   320: 3.471e-01
step   330: 3.450e-01
step   340: 3.427e-01
step   350: 3.402e-01
step   360: 3.375e-01
step   370: 3.346e-01
step   380: 3.314e-01
step   390: 3.280e-01
step   400: 3.243e-01
step   410: 3.203e-01
step   420: 3.161e-01
step   430: 3.116e-01
step   440: 3.067e-01
step   450

In [16]:
optimized_params

array([ 0.37739604,  0.37844712,  0.47128898,  0.88793019,  0.89051222,
        1.01233345, -0.78775134, -0.79029201, -0.88076969, -0.15910703,
       -0.1596567 , -0.22597742,  0.02105887,  0.02105786,  0.02519266,
       -0.02796882])

In [17]:
# calculate solution with initial and optimized parameters for comparison
sol_init = odeint(DiffEqs, y0, t_span, args=(init_params,))
sol_opt = odeint(DiffEqs, y0, t_span, args=(optimized_params,))

## Results 
### On Training Data

In [ ]:
# plot system trajectories
plt.figure(1)
plt.plot(t_span, sol_init[:,:len_c],'--')
plt.gca().set_prop_cycle(None)
plt.plot(t_span, sol_opt[:,:len_c])
plt.gca().set_prop_cycle(None)
plt.plot(t_span, c_exp, 'x')
plt.xlabel('t')
plt.ylabel('c')
plt.legend(('initial c_A', 'initial c_B', 'initial c_X', 
            'final c_A', 'final c_B', 'final c_X'), loc='upper right',ncol=2) # make a legend
plt.grid()
plt.show()

### On New Data

In [37]:
# initial conditions for the concentrations in the reactor
c_A0 = 0.3
c_B0 = 0.7
c_X0 = 0.1

# assemble initial value vector
c0 = [c_A0, c_B0, c_X0]
len_c = len(c0)
len_p = len(init_params)
s0 = np.zeros((len_p*len_c))
y0 = np.concatenate((c0,s0))

In [ ]:
# generate experimental data
# define system equations
def dcdt_2(c, t, params):
    """Mechanistic part of the hybrid model (ODE system describing the time-dependent 
    concentrations in the reactor)"""
    # disassemble input vector
    c_A, c_B, c_X = c
    # calculate reaction rates by neural network prediction
    #r = neural_net_predict(params, c)
    r = 0.08*c_A**0.7*c_B**1.3 # true underlying reaction rate
    # system equations
    dcdt = [(c_Ain-c_A)/tau - r,
            (c_Bin-c_B)/tau - r,
            (c_Xin-c_X)/tau + r]
    return np.array(dcdt)

params = []
sol = odeint(dcdt_2, [c_A0, c_B0, c_X0], t_span, args=(params,))
sol += 0.004*np.random.randn(len(t_span), 3)

c_exp = sol

plt.figure(0)
plt.plot(t_span, sol,'x')
plt.show()

In [39]:
sol_init = odeint(DiffEqs, y0, t_span, args=(init_params,))
sol_opt = odeint(DiffEqs, y0, t_span, args=(optimized_params,))

In [ ]:
# plot system trajectories
plt.figure(1)
plt.plot(t_span, sol_init[:,:len_c],'--')
plt.gca().set_prop_cycle(None)
plt.plot(t_span, sol_opt[:,:len_c])
plt.gca().set_prop_cycle(None)
plt.plot(t_span, c_exp, 'x')
plt.xlabel('t')
plt.ylabel('c')
plt.legend(('initial c_A', 'initial c_B', 'initial c_X', 
            'final c_A', 'final c_B', 'final c_X'), loc='upper right',ncol=2) # make a legend
plt.grid()
plt.savefig("CSTR_experimental_data_and_model_test")
plt.show()